
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# 6 - Ingesting JSON Files with Databricks

In this demonstration, we’ll explore how to ingest JSON files and perform foundational JSON-specific transformations during ingestion, including decoding encoded fields and flattening nested JSON strings. We’ll be working with simulated Kafka event data, sourced from Databricks Marketplace.

### Learning Objectives
By the end of this lesson, you should be able to:
- Ingest raw JSON data into Unity Catalog using CTAS and `read_files()`.
- Apply multiple techniques to flatten JSON string columns with and without converting to a STRUCT type.
- Understand the difference between `explode()` and `explode_outer()`.
- Introduce the capabilities and use cases of the VARIANT data type (public preview as of Q2-2025)

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-06

Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name when writing your tables (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

## B. Overview of CTAS with `read_files()` for Ingestion of JSON files

### B1. Inspect JSON files

1. Run the next cell to verify that there are 11 JSON files located at `/Volumes/dbacademy_ecommerce/v01/raw/events-kafka`.

In [0]:
LIST '/Volumes/dbacademy_ecommerce/v01/raw/events-kafka'

2. Run the cell below to view the raw JSON data in the output. Note the following:

   - Each row contains JSON with 6 key/value pairs.

   - The **key** and **value** fields are encoded in base64. Base64 is an encoding scheme that converts binary data into a readable ASCII string.



<br></br>
**Example Output Formatted**
```
{
    "key": "VUEwMDAwMDAxMDczOTgwNTQ=",
    "offset": 219255030,
    "partition": 0,
    "timestamp": 1593880885085,
    "topic": "clickstream",
    "value": "eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3R
    hbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0sIml0ZW1zIjp
    bXSwidHJhZmZpY19zb3VyY2UiOiJnb29nbGUiLCJ1c2VyX2ZpcnN0X3RvdWNoX3RpbWVzdGFtcCI6MTU5Mzg4MDg4NTA
    zNjEyOSwidXNlcl9pZCI6IlVBMDAwMDAwMTA3Mzk4MDU0In0=",
}
```

In [0]:
SELECT * 
FROM text.`/Volumes/dbacademy_ecommerce/v01/raw/events-kafka`
LIMIT 5;

3. Run the cell below to see how to use `read_files()` to read the JSON data. Notice the following:

   - The JSON file is cleanly read into a tabular format with 6 columns.

   - The **key** and **value** columns are base64-encoded and returned as STRING data type.
   
   - There are no rows in the **_rescued_data** column.

In [0]:
SELECT *
FROM read_files(
  "/Volumes/dbacademy_ecommerce/v01/raw/events-kafka",
  format => "json"
)
LIMIT 10;

### B2. Using CTAS and `read_files()` with JSON

Ingesting JSON files using `read_files()` is as straightforward as reading CSV files.

1. Run the cell below to store this raw data in the **kafka_events_bronze_raw** table and view the table. When inspecting the results, you'll notice that:

   - The **key** and **value** columns are of type STRING and contain data that is **base64-encoded**.

   - This means the actual content has been encoded into base64 format and stored as a string. 
   
   - They have not yet been transformed into a readable string in the first bronze table we create.

**NOTE:** Base64 encoding is commonly used when ingesting data from sources like message queues or streaming platforms, where preserving formatting and avoiding data corruption is important.

In [0]:
-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS kafka_events_bronze_raw;


-- Create the Delta table
CREATE TABLE kafka_events_bronze_raw AS
SELECT *
FROM read_files(
  "/Volumes/dbacademy_ecommerce/v01/raw/events-kafka",
  format => "json"
);


-- Display the table
SELECT *
FROM kafka_events_bronze_raw
LIMIT 10;

### B3. Decoding base64 Strings for the Bronze Table

1. Let's take a look at decoding the **key** and **value** columns by inspecting their data types after applying the `unbase64()` function. The `unbase64` function returns a decoded base64 string as binary.

    - **encoded_key**: The original encoded **key** column as a base64 string.

    - **decoded_key**: A new column created by decoding **key** from a base64 string to BINARY.

    - **encoded_value**: The original encoded **value** column as a base64 string.

    - **decoded_value**: A new column created by decoding **value** from a base64 string to BINARY.

    Run the cell and view the results. Notice that the **decoded_key** and **decoded_value** columns are now BINARY.

In [0]:
SELECT
  key AS encoded_key,
  unbase64(key) AS decoded_key,
  value AS encoded_value,
  unbase64(value) AS decoded_value
FROM kafka_events_bronze_raw
LIMIT 5;

2. Run the next cell to convert the BINARY columns to STRING columns using the `CAST` function. Notice the following in the results:

    - The **decoded_key** and **decoded_value** columns are now of type STRING and readable.

    - The **decoded_value** column is a JSON-formatted string.


In [0]:
SELECT
  key AS encoded_key,
  cast(unbase64(key) AS STRING) AS decoded_key,
  value AS encoded_value,
  cast(unbase64(value) AS STRING) AS decoded_value
FROM kafka_events_bronze_raw
LIMIT 5;

3. Now, let's put it all together to create another bronze-level table named **kafka_events_bronze_decoded**. This table will store the STRING values for the **key** and **value** columns from the original **kafka_events_bronze_raw** table.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_decoded AS
SELECT
  cast(unbase64(key) AS STRING) AS decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  cast(unbase64(value) AS STRING) AS decoded_value
FROM kafka_events_bronze_raw;


-- View the new table
SELECT *
FROM kafka_events_bronze_decoded
LIMIT 5;

## C. Working with JSON Formatted Strings in a Table

### C1. Flattening JSON String Columns

Next, we will explore how extract a column from a column containing a JSON formatted string. 


**BENEFITS**
- **Simple** - Easy to implement and store JSON as plain text.
- **Flexible** - Can hold any JSON structure without schema constraints.

**CONSIDERATIONS**
- **Performance** - STRING columns are slower when querying and processing complex data.
- **No Schema** - The lack of a defined schema for STRING columns can lead to data integrity issues.
- **Complex to Query** - Requires additional code to parse and retrieve data, which can be complex.

#### C1.1 Query JSON strings

You can extract a column from fields containing JSON strings using the syntax: `<column-name>:<extraction-path>`, where `<column-name>` is the string column name and `<extraction-path>` is the path to the field to extract. The returned results are strings. You can also do this with nested fields by using either `.` or `[]`.

This utilizes Spark SQL's built-in functionality to interact directly with nested data stored as JSON strings.

[Query JSON strings](https://docs.databricks.com/aws/en/semi-structured/json)


Example JSON string pulled from a row in the column **decoded_value**:


```
{
    "device": "iOS",
    "ecommerce": {},
    "event_name": "add_item",
    "event_previous_timestamp": 1593880300696751,
    "event_timestamp": 1593880892251310,
    "geo": {
      "city": "Westbrook", 
      "state": "ME"
      },
    "items": [
        {
            "item_id": "M_STAN_T",
            "item_name": "Standard Twin Mattress",
            "item_revenue_in_usd": 595.0,
            "price_in_usd": 595.0,
            "quantity": 1,
        }
    ],
    "traffic_source": "google",
    "user_first_touch_timestamp": 1593880300696751,
    "user_id": "UA000000107392458",
}
```

1. For example, let's extract the following values from the JSON-formatted string:
    - `decoded_value:device`
    - `decoded_value:traffic_source`
    - `decoded_value:geo`
    - `decoded_value:items`

    Run the cell and view the results. Notice that we have successfully extracted the values from the JSON formatted string.

    - **device** is a STRING

    - **traffic_source** is a STRING

    - **geo** is a STRING containing another JSON formatted string
    
    - **item** is a STRING contain an array of JSON formatted strings


In [0]:
SELECT 
  decoded_value,
  decoded_value:device,
  decoded_value:traffic_source,
  decoded_value:geo,       ----- Contains another JSON formatted string
  decoded_value:items      ----- Contains a nested-array of JSON formatted strings
FROM kafka_events_bronze_decoded
LIMIT 5;

2. We can then begin to parse out the necessary JSON formatted string values to create another bronze table to flatten the JSON formatted string column for downstream processing.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_string_flattened AS
SELECT
  decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  decoded_value:device,
  decoded_value:traffic_source,
  decoded_value:geo,       ----- Contains another JSON formatted string
  decoded_value:items      ----- Contains a nested-array of JSON formatted strings
FROM kafka_events_bronze_decoded;


-- Display the table
SELECT *
FROM kafka_events_bronze_string_flattened;

### C2. Flattening JSON Formatting Strings via STRUCT Conversion

Similar to the previous section, we will discuss how to flatten our JSON STRING column **decoded_value** using a STRUCT column.

#### Benefits and Considerations of STRUCT Columns

**Benefits**
- **Schema Enforcement** – STRUCT columns define and enforce a schema, helping maintain data integrity.
- **Improved Performance** – STRUCTs are generally more efficient for querying and processing than plain strings.

**Considerations**
- **Schema Enforcement** – Because the schema is enforced, issues can arise if the JSON structure changes over time.
- **Reduced Flexibility** – The data must consistently match the defined schema, leaving less room for structural variation.

#### C2.1 Converting a JSON STRING to a STRUCT Column
To convert a JSON-formatted STRING column to a STRUCT column, you will need to derive the schema of the JSON-formatted string and then parse each row into a STRUCT type.

We can do this in two steps.
  1. Get the STRUCT type of the JSON formatted string.
  2. Apply the STRUCT to the JSON formatted string column.

**NOTE:** We have already copied and pasted the correct values for you as a part of this demonstration. The subsequent cell below is a copy and paste of the output of the single row that appears when running the next cell. 


1. Determine the derived schema using the [**`schema_of_json()`**](https://docs.databricks.com/en/sql/language-manual/functions/schema_of_json.html) function, which returns the schema inferred from a JSON-formatted string.

    Run the cell and view the results. Notice that the output displays the structure of the JSON string.

In [0]:
SELECT schema_of_json('{"device":"Linux","ecommerce":{"purchase_revenue_in_usd":1075.5,"total_item_quantity":1,"unique_items":1},"event_name":"finalize","event_previous_timestamp":1593879231210816,"event_timestamp":1593879335779563,"geo":{"city":"Houston","state":"TX"},"items":[{"coupon":"NEWBED10","item_id":"M_STAN_K","item_name":"Standard King Mattress","item_revenue_in_usd":1075.5,"price_in_usd":1195.0,"quantity":1}],"traffic_source":"email","user_first_touch_timestamp":1593454417513109,"user_id":"UA000000106116176"}')
AS schema

2. Copy and paste the output from `schema_of_json` into the [**`from_json()`**](https://docs.databricks.com/en/sql/language-manual/functions/from_json.html) function. This function parses a column containing a JSON-formatted string into a STRUCT type using the specified schema, and creates a new table named **kafka_events_bronze_struct**.

    Run the cell and view the results. Notice that the **value** column has been transformed into a nested STRUCT that includes scalar fields, nested structs, and an array of structs.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_struct AS
SELECT 
  * EXCEPT (decoded_value),
  from_json(
      decoded_value,    -- JSON formatted string column
      'STRUCT<device: STRING, ecommerce: STRUCT<purchase_revenue_in_usd: DOUBLE, total_item_quantity: BIGINT, unique_items: BIGINT>, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>') AS value
FROM kafka_events_bronze_decoded;


-- View the new table.
SELECT *
FROM kafka_events_bronze_struct
LIMIT 5;

#### C2.2 Extract fields, nested fields, and nested arrays from STRUCT columns

We can query the STRUCT column using `value.device` or `value.ecommerce` in our SELECT statement.

1. Using this syntax, we can obtain values from the **value** struct column. Run the cell and view the results. Notice the following:

    - We obtained values from the STRUCT column for **device** and **city**
    
    - The **items** column contains an ARRAY of STRUCTS. The number of elements in the array varies.

In [0]:
SELECT 
  decoded_key,
  value.device as device,  -- <----- Field
  value.geo.city as city,  -- <----- Nested-field from geo field
  value.items as items,
  array_size(items) AS number_elements_in_array -- <----- Count the number of elements in the array column items
FROM kafka_events_bronze_struct
ORDER BY number_elements_in_array DESC;

#### C2.3 Explode Arrays

Exploding an array transforms each element of an array column into a separate row, effectively flattening the array. There are a few things to keep in mind when using this function. 

1. It returns a set of rows composed of the elements of the array or the keys and values of the map.

1. If the array is `NULL` no rows are produced. To return a single row with `NULL`s for the array or map values use the [`explode_outer()`](https://docs.databricks.com/gcp/en/sql/language-manual/functions/explode_outer) function.

1. Run the cell to see how the ARRAY of values in the `value.items` explodes the array into one row for each element in the array.

In [0]:
CREATE OR REPLACE TABLE bronze_explode_array AS
SELECT
  decoded_key,
  array_size(value.items) AS number_elements_in_array,
  explode(value.items) AS item_in_array,
  value.items
FROM kafka_events_bronze_struct
ORDER BY number_elements_in_array DESC;


-- Display table
SELECT *
FROM bronze_explode_array;

## D. Working with a VARIANT Column (Public Preview)

#### VARIANT Column Benefits and Considerations:

**BENEFITS**
- **Open** - Fully open-sourced, no proprietary data lock-in.
- **Flexible** - No strict schema. You can put any type of semi-structured data into VARIANT.
- **Performant** - Improved performance over existing methods.

**CONSIDERATIONS**
- Currently in public preview as of 2025 Q2.
- [Variant support in Delta Lake](https://docs.databricks.com/aws/en/delta/variant)

**RESOURCES**:
- [Introducing the Open Variant Data Type in Delta Lake and Apache Spark](https://www.databricks.com/blog/introducing-open-variant-data-type-delta-lake-and-apache-spark)
- [Say goodbye to messy JSON headaches with VARIANT](https://www.youtube.com/watch?v=fWdxF7nL3YI)
- [Variant Data Type - Making Semi-Structured Data Fast and Simple](https://www.youtube.com/watch?v=jtjOfggD4YY)


**NOTE:** Variant data type will not work on Serverless Version 1.

1. View the **kafka_events_bronze_decoded** table. Confirm the **decoded_value** column contains a JSON formatted string.

In [0]:
SELECT *
FROM kafka_events_bronze_decoded
LIMIT 5;

2. Use the [`parse_json`](https://docs.databricks.com/aws/en/sql/language-manual/functions/parse_json) function to returns a VARIANT value from the JSON formatted string.

    Run the cell and view the results. Notice that the **json_variant_value** column is of type VARIANT.

In [0]:
CREATE OR REPLACE TABLE kafka_events_bronze_variant AS
SELECT
  decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  parse_json(decoded_value) AS json_variant_value   -- Convert the decoded_value column to a variant data type
FROM kafka_events_bronze_decoded;

-- View the table
SELECT *
FROM kafka_events_bronze_variant
LIMIT 5;

3. You can parse the VARIANT data type column using `:` to create your desired table.

    [VARIANT type](https://docs.databricks.com/aws/en/sql/language-manual/data-types/variant-type)

In [0]:
SELECT
  json_variant_value,
  json_variant_value:device :: STRING,  -- Obtain the value of device and cast to a string
  json_variant_value:items
FROM kafka_events_bronze_variant
LIMIT 10;


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>